## Spotify Docs: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/
## Notações:
<strong>duration_ms:</strong> Duração da música<br>
<strong>key:</strong> Afinação (0 = C, 1 = C♯/D♭, 2 = D). Default = -1<br>
<strong>mode:</strong> Escala musical (maior ou menor)<br>
<strong>time_signature:</strong> quantas batidas possui em cada barra ex: 3/4<br>
<strong>acousticness:</strong> confiabilidade da música ser acústica 1= certeza 0 =  não acústica<br>
<strong>danceability:</strong> o quão dançável a música é: 1=mto 0=nada<br>
<strong>energy:</strong> intencidade da música (rápidas, altas e barulhentas)<br>
<strong>instrumentalness:</strong> conteúdo vocal na música: 1=sem vocal 0=mto vocal<br>
<strong>liveness:</strong> chance da música ser ao vivo: 0.8>= grande chance de ser ao vivo<br>
<strong>loudness:</strong> valores decibéis (dB) da música<br>
<strong>speechiness:</strong> Quanto mais próximo de 1 maior a chance de ser podcast, audiobook ou poesia 0.33 >= && 0.66< (maior chance de vocal com música) <0.33 músicas quase ou sem vocal<br>
<strong>valence:</strong> Quanto mais próxima de 1 mais alegra e quanto mais próximo de 0 é mais triste<br>
<strong>tempo:</strong> BPMN

In [213]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

# with open("songDb.tsv", 'r') as myfile:
#     with open("songDb.csv", 'w') as csv_file:
#         for line in myfile:
#             fileContent = re.sub("\t", ",", line)
#             csv_file.write(fileContent)
        
#Load dataset
playlistSpotify = pd.read_csv('songDb.csv', dtype={},encoding='ISO-8859-1')
# playlistSpotify = pd.read_csv('short_songDb.csv', dtype={},encoding='ISO-8859-1')
playlistSpotify.head()

C:\Users\Lucas\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (11,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Name,Danceability,Energy,Key,Loudness,Mode,Speechness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Type,ID,Uri,Ref_Track,URL_features,Duration_ms,time_signature,Genre
0,YuveYuveYu,0.624,0.857,10.0,-6.250,0.0,0.0542,0.020800,0.206000,0.110,0.324,131.926,audio_features,6J2VvzKwWc2f0JP5RQVZjq,spotify:track:6J2VvzKwWc2f0JP5RQVZjq,https://api.spotify.com/v1/tracks/6J2VvzKwWc2f...,https://api.spotify.com/v1/audio-analysis/6J2V...,282920.0,4.0,celticmetal
1,Gloryhammer,0.517,0.916,0.0,-4.933,1.0,0.0559,0.000182,0.001910,0.306,0.444,135.996,audio_features,4HA34COgxgVJ6zK88UN4Ik,spotify:track:4HA34COgxgVJ6zK88UN4Ik,https://api.spotify.com/v1/tracks/4HA34COgxgVJ...,https://api.spotify.com/v1/audio-analysis/4HA3...,300320.0,4.0,celticmetal
2,Nostos,0.251,0.894,8.0,-4.103,0.0,0.0570,0.014400,0.000000,0.123,0.297,114.223,audio_features,3W6Xik6Xxf06JuUoZSATlD,spotify:track:3W6Xik6Xxf06JuUoZSATlD,https://api.spotify.com/v1/tracks/3W6Xik6Xxf06...,https://api.spotify.com/v1/audio-analysis/3W6X...,175353.0,4.0,celticmetal
3,Yggdrasil,0.469,0.743,1.0,-5.570,0.0,0.0272,0.002220,0.000111,0.276,0.481,86.953,audio_features,2gGveBaLJQMtJ43X4UL5kH,spotify:track:2gGveBaLJQMtJ43X4UL5kH,https://api.spotify.com/v1/tracks/2gGveBaLJQMt...,https://api.spotify.com/v1/audio-analysis/2gGv...,272292.0,4.0,celticmetal
4,Incense&Iron,0.487,0.952,1.0,-4.429,0.0,0.0613,0.000228,0.000000,0.161,0.329,125.993,audio_features,1lRF81A1C9QoCgBcEop2zg,spotify:track:1lRF81A1C9QoCgBcEop2zg,https://api.spotify.com/v1/tracks/1lRF81A1C9Qo...,https://api.spotify.com/v1/audio-analysis/1lRF...,237933.0,4.0,celticmetal


In [230]:
playlist = [] #array com dados formatados
speechiness = 0
for key, songData in playlistSpotify.iterrows():
#     print('songData.Speechness', songData.Speechness)
    if(songData.Speechness < 0.33):
        speechiness = 0
    elif (songData.Speechness >= 0.33 and songData.Speechness < 0.66):
#         print('musica e fala', songData.Speechness)
        speechiness = 1
    else:
        speechiness = 2
#         print('somente fala', songData.Speechness)
        
#     print('speechiness',speechiness)
    playlist.append([
        songData.Danceability,
        songData.Energy,
        songData.Loudness,
        speechiness,# songData.Speechness,
        songData.Acousticness,
        songData.Instrumentalness,
        songData.Liveness,# 1 if songData.Liveness > 0.8 else 0,
        songData.Valence,
        songData.Tempo
    ])

print('ok')
# print(playlist)
# print('\n'.join('{}: {}'.format(*k) for k in enumerate(playlist)))

ok


In [231]:
# Atributos e classes
pdPlaylist = pd.DataFrame(playlist)

X = pdPlaylist.iloc[:, 0:8]
y = pdPlaylist.iloc[: ,3]
# y = pdPlaylist.iloc[: ,6]

y = y.astype('int')


print(y)
print(X)

0         0
1         0
2         0
3         0
4         0
         ..
131575    0
131576    0
131577    0
131578    0
131579    0
Name: 3, Length: 131580, dtype: int32
            0      1      2  3         4         5      6      7
0       0.624  0.857 -6.250  0  0.020800  0.206000  0.110  0.324
1       0.517  0.916 -4.933  0  0.000182  0.001910  0.306  0.444
2       0.251  0.894 -4.103  0  0.014400  0.000000  0.123  0.297
3       0.469  0.743 -5.570  0  0.002220  0.000111  0.276  0.481
4       0.487  0.952 -4.429  0  0.000228  0.000000  0.161  0.329
...       ...    ...    ... ..       ...       ...    ...    ...
131575  0.568  0.708 -9.960  0  0.007930  0.000528  0.266  0.214
131576  0.470  0.909 -1.674  0  0.061100  0.000000  0.294  0.607
131577  0.272  0.918 -2.589  0  0.000749  0.009200  0.307  0.530
131578  0.402  0.902 -4.115  0  0.000073  0.004650  0.261  0.569
131579  0.482  0.839 -4.171  0  0.013400  0.244000  0.104  0.571

[131580 rows x 8 columns]


In [232]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [233]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X_train)

# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [234]:
# print(X_test)

In [235]:
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)
mlp.fit(X_train, y_train.values.ravel())

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 10, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [242]:
predictions = mlp.predict(X_test)

In [243]:
# print(classification_report(y_test,predictions,zero_division=0)) 
# print(classification_report(y_test,predictions,zero_division=0))

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[38211     0     0]
 [    0  1031     0]
 [    0     3   229]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     38211
           1       1.00      1.00      1.00      1031
           2       1.00      0.99      0.99       232

    accuracy                           1.00     39474
   macro avg       1.00      1.00      1.00     39474
weighted avg       1.00      1.00      1.00     39474



In [244]:
#Armazena as predicões
predictions = cross_val_predict(mlp,X,y,cv=4)
#Calcula acurácia do treino
accuracy_score(y,predictions)*100

99.98784009727922

In [245]:
#Gera a matriz de confusão do treino
confusion_matrix(y,predictions)

array([[127357,      7,      0],
       [     4,   3376,      2],
       [     1,      2,    831]], dtype=int64)

In [246]:
#Gera a matriz de confusão do test
print(classification_report(y,predictions,zero_division=0))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    127364
           1       1.00      1.00      1.00      3382
           2       1.00      1.00      1.00       834

    accuracy                           1.00    131580
   macro avg       1.00      1.00      1.00    131580
weighted avg       1.00      1.00      1.00    131580

